<a href="https://colab.research.google.com/github/bkrant/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/DS_Unit_1_Sprint_Challenge_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [0]:
import pandas as pd

cell_phones = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv')

population = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv')

geo_country_codes = (pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')
                       .rename(columns={'country': 'geo', 'name': 'country'}))

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [5]:
print(population.shape)
population.head(2)

(59297, 3)


,geo,time,population_total
0,afg,1800,3280000
1,afg,1801,3280000


In [6]:
print(cell_phones.shape)
cell_phones.head(2)

(9215, 3)


,geo,time,cell_phones_total
0,abw,1960,0.0
1,abw,1965,0.0


In [9]:
print(geo_country_codes.shape)
geo_country_codes.head(2)

(273, 33)


,geo,alt_5,alternative_1,alternative_2,alternative_3,alternative_4_cdiac,arb1,arb2,arb3,arb4,...,latitude,longitude,main_religion_2008,country,pandg,un_state,unicode_region_subtag,upper_case_name,world_4region,world_6region
0,abkh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Abkhazia,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,NaN,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,...,12.5,-69.96667,christian,Aruba,NaN,False,AW,ARUBA,americas,america


In [0]:
merge1 = pd.merge(cell_phones, population, how='inner', on=['geo', 'time'])

In [10]:
print(merge1.shape)
merge1.head(2)

(8590, 4)


,geo,time,cell_phones_total,population_total
0,afg,1960,0.0,8996351
1,afg,1965,0.0,9938414


Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

In [24]:
df = pd.merge(geo_country_codes[['geo', 'country']], merge1, on='geo', how='right')
print(df.shape)
df.head(2)

(8590, 5)


,geo,country,time,cell_phones_total,population_total
0,afg,Afghanistan,1960,0.0,8996351
1,afg,Afghanistan,1965,0.0,9938414


## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [0]:
df['cell/person'] = df['cell_phones_total']/df['population_total']

In [26]:
df.head(2)

,geo,country,time,cell_phones_total,population_total,cell/person
0,afg,Afghanistan,1960,0.0,8996351,0.0
1,afg,Afghanistan,1965,0.0,9938414,0.0


In [51]:
df.groupby(['country','time']).mean().unstack().head(2)

cell_phones_total                                               \
time                     1960 1965 1970 1975 1976 1977 1978 1979 1980 1981   
country                                                                      
Afghanistan               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Albania                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

               ...    cell/person                                          \
time           ...           2008      2009      2010      2011      2012   
country        ...                                                          
Afghanistan    ...       0.289401  0.374942  0.354678  0.464441  0.499728   
Albania        ...       0.621607  0.831605  0.915609  1.059228  1.198614   

                                                               
time             2013      2014      2015      2016      2017  
country                                                        
Afghanistan  0.529665  0.561913  0.584205  0.623354  0.673506  
Albania      1.262765  1.150261  1.163375  1.151523  1.193763  

[2 rows x 138 columns]

In [49]:
df.groupby(['country','time']).mean().unstack()['cell/person'][2017]['United States']

1.2201246847283354

Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [53]:
df['geo'] = df['geo'].str.upper()
df['geo'].head()

0    AFG
1    AFG
2    AFG
3    AFG
4    AFG
Name: geo, dtype: object

## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [54]:
df.describe()

,time,cell_phones_total,population_total,cell/person
count,8590.000000,8.590000e+03,8.590000e+03,8590.000000
mean,1994.193481,9.004950e+06,2.983823e+07,0.279639
std,14.257975,5.573408e+07,1.161284e+08,0.454247
min,1960.000000,0.000000e+00,4.433000e+03,0.000000
25%,1983.000000,0.000000e+00,1.456148e+06,0.000000
50%,1995.000000,6.200000e+03,5.725062e+06,0.001564
75%,2006.000000,1.697652e+06,1.810581e+07,0.461149
max,2017.000000,1.474097e+09,1.409517e+09,2.490243


In [57]:
df.describe(exclude='number')

,geo,country
count,8590,8590
unique,195,195
top,IRN,Bhutan
freq,46,46


In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [0]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

In [66]:
df.columns.tolist()

['geo',
 'country',
 'time',
 'cell_phones_total',
 'population_total',
 'cell/person']

In [119]:
df[df['time']==2017].groupby('country')['cell_phones_total'].sum().sort_values(ascending=False).head()

country
China           1,474,097,000.0
India           1,168,902,277.0
Indonesia         458,923,202.0
United States     395,881,000.0
Brazil            236,488,548.0
Name: cell_phones_total, dtype: float64

In [120]:
df[df['time']==2017].pivot_table(index='country',aggfunc=sum).sort_values('cell_phones_total', ascending=False)['cell_phones_total'].head()

country
China           1,474,097,000.0
India           1,168,902,277.0
Indonesia         458,923,202.0
United States     395,881,000.0
Brazil            236,488,548.0
Name: cell_phones_total, dtype: float64

2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

In [149]:
us.cell_phones_total[0], us.population_total[0], len(us)

(0.0, 186808228, 43)

In [150]:
us.loc[(slice(None), [1960]), :]

,,cell_phones_total,population_total
country,time,,
United States,1960,0.0,186808228


In [0]:
us = df[df['country']=='United States'].pivot_table(index=['country','time'],aggfunc=sum)[['cell_phones_total','population_total']]

In [157]:
us[:2]

cell_phones_total  population_total
country       time                                     
United States 1960                0.0         186808228
              1965                0.0         199815540

In [156]:
for i in range(len(us)):
  if us.cell_phones_total[i] > us.population_total[i]:
    print(us.loc[(slice(None), [i+1960]), :])
    break

                    cell_phones_total  population_total
country       time                                     
United States 1999       86,047,003.0         278862277


## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

In [159]:
df.head(2)

,geo,country,time,cell_phones_total,population_total,cell/person
0,AFG,Afghanistan,1960,0.0,8996351,0.0
1,AFG,Afghanistan,1965,0.0,9938414,0.0


In [161]:
df.pivot_table(index='time', columns='country', values='cell_phones_total')

country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
time,,,,,,,,,,,,,,,,,,,,,
1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

If you have the time and curiosity, what other questions can you ask and answer with this data?